In [2]:
#?????
! wc segmented_train_seg_by_word.txt   # in this txt file the chinese lines are in the format of word + space

 20000000 241088714 1191619649 segmented_train_seg_by_word.txt


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

import collections
from collections import Counter
from utils import ProgressBar # show the process of a long running operation

import keras.preprocessing.sequence

%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
enline = None
chline = None

sentlength = 4  # sentence less than 4 words

enlines = []
chlines = []

#pb = progressbar.ProgressBar(worksum = 100000000)
#pb.startjob()
num = 0
with open("segmented_train_seg_by_word.txt") as fhdl:
    for line in fhdl:
        num += 1
        if num % 2 == 1:
            enline = line
            continue
        else:
            chline = line
        
        # split english line and cliense line to words list
        enlinesp = [word.lower() for word in enline.strip("\n").split()]
        chlinesp = [word for word in chline.strip("\n").split()]
        
        # only use sentences <= 4 words to reduce running time
        if  len(enlinesp) <= sentlength and len(chlinesp) <= sentlength:
            enlines.append(enlinesp)
            chlines.append(chlinesp)
            
        # log the progress when reachine 2000
        if (num//2) % 10000 == 0:
            #pb.complete(1000)
            #print(num)
            pass

In [15]:
len(enlines), len(chlines)

(150904, 150904)

In [16]:
enlines[:10]

[['deuces', 'the', 'winner', '.'],
 ['one', 'against', '500', '.'],
 ['a', 'twin', '.'],
 ['twin', 'girls', '.'],
 ['identical', 'twins', '.'],
 ['pair', 'of', 'underachievers', '.'],
 ['husband', 'and', 'wife', '.'],
 ['couple', '.'],
 ['nice', 'couple', '.'],
 ['a', 'young', 'couple', '.']]

In [17]:
enlines[:10]

[['deuces', 'the', 'winner', '.'],
 ['one', 'against', '500', '.'],
 ['a', 'twin', '.'],
 ['twin', 'girls', '.'],
 ['identical', 'twins', '.'],
 ['pair', 'of', 'underachievers', '.'],
 ['husband', 'and', 'wife', '.'],
 ['couple', '.'],
 ['nice', 'couple', '.'],
 ['a', 'young', 'couple', '.']]

In [18]:
words = []

# put all the chinese and english word lists to one word list
# to statistic all the words
for sent in chlines:
    for word in sent:
        words.append(word)

for sent in enlines:
    for word in sent:
        words.append(word)

In [19]:
from collections import Counter

In [20]:
Counter(words).most_common(15)

[('.', 69213),
 ('。', 64997),
 ('？', 18494),
 ('?', 18486),
 ('的', 15156),
 ('...', 15073),
 (',', 13814),
 ('，', 12598),
 ('the', 11865),
 ('我', 11374),
 ('！', 11165),
 ('!', 10976),
 ('i', 8885),
 ('你', 7707),
 ('了', 7633)]

In [21]:
#??????????????
def addchar(what2ind, ind2what, c):
    if c in what2ind:
        return
    ind2what[len(what2ind)] = c
    what2ind[c] = len(what2ind)

In [22]:
# build a dictionary
# i  1
# am 2
# teacher <unk>

word2ind = {}    # dictionary from word to index
ind2word = {}    # dictionary from index to word

specialchars = ["<pad>", "<unk>"]

for one in specialchars:
    addchar(word2ind, ind2word, one)

In [28]:
for word,_ in Counter(words).most_common(10000):   # add the 1000 most common words into the dictionary
    addchar(word2ind,ind2word,word)

In [29]:
dat_x_in = []
dat_y    = []

for chsent, ensent in zip(chlines, enlines):
    indsent = [word2ind.get(i, word2ind["<unk>"]) for i in chsent]
    dat_x_in.append(indsent)
    dat_y.append(0)
    
    indsent = [word2ind.get(i, word2ind["<unk>"]) for i in ensent]
    dat_x_in.append(indsent)
    dat_y.append(1)

In [33]:
dat_x_in = keras.preprocessing.sequence.pad_sequences(dat_x_in,
                                                        padding = "post",
                                                        value = word2ind["<pad>"])


In [34]:
dat_y = np.asarray(dat_y)

In [35]:
dat_x_in.shape, dat_y.shape

((301808, 4), (301808,))

In [36]:
dat_x_in[:10]

array([[   1,    1,    3,    0],
       [   1,   10, 5685,    2],
       [2188,    1,    1,    3],
       [  62, 1053, 4818,    2],
       [2188, 3712,    3,    0],
       [  17, 4273,    2,    0],
       [2188, 3712,  589,    3],
       [4273,  761,    2,    0],
       [2188,    1, 3712,    3],
       [8988, 3941,    2,    0]], dtype=int32)

In [38]:
type(dat_x_in), type(dat_y)

(numpy.ndarray, numpy.ndarray)

In [39]:
def data_generator(batch_size):
    # array = []
    # delete while True
    while True:  # why not infinite loop
        for i in range(0, len(dat_x_in), batch_size):
            if i + batch_size <len(dat_x_in):
                yield dat_x_in[i:i + batch_size], dat_y[i:i + batch_size] 
                # array.append()
                # create batches and hannging them there
                #[n, batch_size, 4]

In [40]:
gen = data_generator(256)   # batch size 256

In [41]:
batch_x, batch_y = gen.__next__()  # everytime run the generator next, run one iteration till the yield, then stop

In [42]:
batch_x

array([[   1,    1,    3,    0],
       [   1,   10, 5685,    2],
       [2188,    1,    1,    3],
       ...,
       [3791,    1,    2,    0],
       [2564,    1,    4,    0],
       [  17, 4894,    5,    0]], dtype=int32)

In [43]:
batch_y

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [44]:
batch_x.shape, batch_y.shape

((256, 4), (256,))

# model part

In [45]:
batch_size = 256
embedding_size = 100
vocabulary_size = len(ind2word)
num_units = 50

In [46]:
import tensorflow as tf
import tflearn

ModuleNotFoundError: No module named 'tflearn'

In [50]:
tf.reset_default_graph()
config = tf.ConfigProto(log_device_placement=True, 
                        allow_soft_placement=True )
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [54]:
with tf.device("/gpu:1"): 
    initializer = tf.random_uniform_initializer(-0.08, 0.08)
    tf.get_variable_scope().set_initializer(initializer)
    
    x = tf.placeholder("int32", [None, None])   #[batch_size, 句子长度] 二者都可调
    y = tf.placeholder("int32", None)
    x_len = tf.placeholder("int32", [None])
    
    learning_rate = tf.placeholder(tf.float32, shape=[])
    
    # word embedding
    embedding_encoder = tf.get_variable( "embedding_encoder",
                                       [vocabulary_size, embedding_size],     #【10000， 100】
                                       dtype = tf.float32)
    encoder_emb_inp = tf.nn.embedding_lookup(embedding_encoder, x)
    
    encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)
    
    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell,      # 可变长的rnn 可以支持增加rnn的个数
                                                    encoder_emb_inp,
                                                    sequence_length = x_len,
                                                    time_major=False,
                                                    dtype = tf.float32)
    
    # fully connection layer fully connect to one classifiler
    model_logistic = tf.layers.dense(encoder_state[0], 1)
    model_pred = tf.nn.sigmoid(model_logistic)
    
    loss = tf.nn.sigmoid_cross_entropy_with_logits(
        labels = tf.cast(y,tf.float32),
        logits = tf.reshape(model_logistic, (-1,)))
    loss = tf.reduce_mean(loss)
    optimizer = tf.train.GradientDescentOptimizer(
        learning_rate = learning_rate).minimize(loss)

'\nwith tf.device("/gpu:1"): # ?????\n    initializer = tf.random_uniform_initializer(-0.08, 0.08)\n    tf.get_variable_scope().set_initializer(initializer)\n    \n    x = tf.placeholder("int32", [None, None])\n    y = tf.placeholder("int32", None)\n    x_len = tf.placeholder("int32", [None])\n    \n    learning_rate = tf.placeholder(tf.float32, shape=[])\n    \n    embedding_encoder = tf.get_variable( "embedding_encoder",\n                                       [vocabulary_size, embedding_size],\n                                       dtype = tf.float32)\n    encoder_emb_inp = tf.nn.embedding_lookup(embedding_encoder, x)\n    \n    encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)\n    \n    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell,\n                                                    encoder_emb_inp,\n                                                    sequence_length = x_len,\n                                                    time_major=False,\n        

In [55]:
session.run(tf.global_variables_initializer())  # Returns an Op that initializes global variables.

In [56]:
losses = []
beginning_lr = 0.1
gen = data_generator(batch_size)

# 只运行一个epoch
for one_epoch in range(0, 1):
    for one_batch in range(1, len(dat_x_in), batch_size):
        batch_x, batch_y = gen.__next__()
        batch_x_len = np.asarray([len(i) for i in batch_x])
        batch_lr = beginning_lr
        
        _, batch_loss = session.run([optimizer, loss], 
                                    feed_dict={
                                                x:batch_x,
                                                y:batch_y,
                                                x_len:batch_x_len,
                                                learning_rate:batch_lr,
                                               }
                                   )
        losses.append(batch_loss)

NameError: name 'optimizer' is not defined

In [57]:
matplotlib inline
pd.Dataframe(losses).plot()

SyntaxError: invalid syntax (<ipython-input-57-65792a5c845e>, line 1)

In [58]:
def predict_result(sent):
    senttoken = [word2ind[i.lower()] for i in sent.split()]
    inputx = np.asarray([senttoken])
    inputx_len = np.asarray([len(senttoken)])
    
    print(inputx, inputx_len)
    batch_predict = session.run(model_pred,
                               feed_dict = {
                                   x:inputx,
                                   x_len:inputx_len,
                               })[0]
    return “英文” if batch_predict > 0.5 else "中文"

SyntaxError: invalid character in identifier (<ipython-input-58-566bed6efba9>, line 12)

In [59]:
source_sentence = "who are you, what do you want ."

In [ ]:
predict_result(source_sentence)